In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm import tqdm
from scipy.optimize import minimize
from skfda import FDataGrid
from skfda.preprocessing.smoothing import BasisSmoother
from skfda.representation.basis import FourierBasis
from skfda.exploratory.visualization import FPCAPlot
from sklearn.cluster import KMeans
from sklearn.pipeline import make_pipeline
from skfda.preprocessing.dim_reduction.feature_extraction import FPCA
from joblib import Parallel, delayed
from tqdm import tqdm

In [4]:
data_dir='levelups_panel_1004_final.csv'
data=pd.read_csv(data_dir, encoding=False)
data

,player_slug,character_slug,first_session_level,binger,total_purchase,total_ad,highest_level,status,level_from,level_to,...,prev_interval,interval_ratio,interval_until,realtime_tot,level_to_by_10,time_diff,more_than_3_hours,total_num_played,interval_tot,rounded_ratio
0,2021-11-01T10:41:53.949Z,화요,119.0,1,0.0,28.0,139,complete,1,2,...,0.010667,1.000000,0.020556,3.726111,"(0, 10]",0 days 00:00:14,False,0,3.726111,1.0
1,2021-11-01T10:41:53.949Z,화요,119.0,1,0.0,28.0,139,complete,2,3,...,0.020556,0.202703,0.024722,3.726111,"(0, 10]",0 days 00:00:21,False,0,3.726111,1.0
2,2021-11-01T10:41:53.949Z,화요,119.0,1,0.0,28.0,139,complete,3,4,...,0.004167,1.466667,0.030833,3.726111,"(0, 10]",0 days 00:00:23,False,0,3.726111,1.0
3,2021-11-01T10:41:53.949Z,화요,119.0,1,0.0,28.0,139,complete,4,5,...,0.006111,1.090909,0.037500,3.726111,"(0, 10]",0 days 00:01:10,False,0,3.726111,1.0
4,2021-11-01T10:41:53.949Z,화요,119.0,1,0.0,28.0,139,complete,5,6,...,0.006667,2.958333,0.057222,3.726111,"(0, 10]",0 days 00:01:16,False,0,3.726111,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
710815,2022-03-30T06:38:53.434Z,뿌숑,115.0,1,0.0,1.0,115,complete,95,96,...,0.059444,0.098131,2.621667,2.818333,"(90, 100]",0 days 00:00:10,False,0,2.818333,1.0
710816,2022-03-30T06:38:53.434Z,뿌숑,115.0,1,0.0,1.0,115,complete,96,97,...,0.005833,0.523810,2.624722,2.818333,"(90, 100]",0 days 00:02:50,False,0,2.818333,1.0
710817,2022-03-30T06:38:53.434Z,뿌숑,115.0,1,0.0,1.0,115,complete,97,98,...,0.003056,15.545455,2.672222,2.818333,"(90, 100]",0 days 00:04:19,False,0,2.818333,1.0
710818,2022-03-30T06:38:53.434Z,뿌숑,115.0,1,0.0,1.0,115,complete,98,99,...,0.047500,1.520468,2.744444,2.818333,"(90, 100]",0 days 00:04:25,False,0,2.818333,1.0


In [7]:
binge=data['rounded_ratio'].quantile(0.80)
binge

0.89

In [11]:
data['binge'] = data.apply(lambda row: 1 if row['rounded_ratio'] >= binge else 0, axis=1)

In [13]:
data_last= data.drop_duplicates(subset='player_slug', keep='last')
data_last

,player_slug,character_slug,first_session_level,binger,total_purchase,total_ad,highest_level,status,level_from,level_to,...,prev_interval,interval_ratio,interval_until,realtime_tot,level_to_by_10,time_diff,more_than_3_hours,total_num_played,interval_tot,rounded_ratio
98,2021-11-01T10:41:53.949Z,화요,119.0,1,0.0,28.0,139,complete,99,100,...,0.401944,1.280581,3.726111,3.726111,"(90, 100]",NaN,False,0,3.726111,1.00
197,2021-11-01T14:12:34.729Z,레서판다,71.0,1,0.0,0.0,102,complete,99,100,...,0.002222,0.875000,4.596889,13.554167,"(90, 100]",NaN,False,1,4.596889,0.34
296,2021-11-01T15:01:02.831Z,벤쿤,10.0,0,0.0,0.0,129,complete,99,100,...,0.121111,1.119266,4.739389,22.170833,"(90, 100]",NaN,False,2,4.739389,0.21
395,2021-11-01T15:10:02.121Z,딸건,8.0,0,3300.0,2.0,115,complete,99,100,...,0.157778,1.588028,5.613056,130.908889,"(90, 100]",NaN,False,2,5.613056,0.04
494,2021-11-01T15:16:50.510Z,아픈발꾸락,20.0,0,0.0,85.0,123,complete,99,100,...,0.051389,218.708108,12.723278,91.696667,"(90, 100]",NaN,False,5,12.723278,0.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
710423,2022-03-28T00:01:33.666Z,벤쿱,99.0,1,3300.0,2.0,103,complete,99,100,...,0.174722,3.438792,5.423167,6.873611,"(90, 100]",NaN,False,0,5.423167,0.79
710522,2022-03-28T07:44:40.201Z,도오옹건,87.0,1,0.0,0.0,139,complete,99,100,...,0.043056,1.567742,8.681944,27.251944,"(90, 100]",NaN,False,3,8.681944,0.32
710621,2022-03-28T11:06:23.113Z,구또,47.0,0,122100.0,0.0,124,complete,99,100,...,0.172222,0.869355,2.715056,3.178889,"(90, 100]",NaN,False,0,2.715056,0.85
710720,2022-03-29T09:29:33.653Z,이가탄,7.0,0,0.0,6.0,120,complete,99,100,...,0.093333,1.101190,2.575833,23.934722,"(90, 100]",NaN,False,2,2.575833,0.11


In [21]:
data_last[(data_last['status']=='drop') & (data_last['binge']==1)]

,player_slug,character_slug,first_session_level,binger,total_purchase,total_ad,highest_level,status,level_from,level_to,...,prev_interval,interval_ratio,interval_until,realtime_tot,level_to_by_10,time_diff,more_than_3_hours,total_num_played,interval_tot,rounded_ratio
1286,2021-11-02T11:20:22.086Z,투볍,100.0,1,0.0,8.0,100,drop,99,100,...,0.350000,0.991270,4.604722,4.604722,"(90, 100]",NaN,False,0,4.604722,1.00
2573,2021-11-04T14:17:04.890Z,보탱이,98.0,1,0.0,2.0,100,drop,99,100,...,0.154444,33.705036,8.216389,8.216389,"(90, 100]",NaN,False,1,8.216389,1.00
22868,2021-11-26T04:50:08.676Z,대천사,100.0,1,0.0,0.0,100,drop,99,100,...,0.162500,1.030769,4.998056,4.998056,"(90, 100]",NaN,False,0,4.998056,1.00
28214,2021-12-03T19:43:46.891Z,ARLN,89.0,1,0.0,0.0,100,drop,99,100,...,0.298889,8.500000,8.943611,8.943611,"(90, 100]",NaN,False,1,8.943611,1.00
54548,2021-12-18T02:38:05.416Z,오도리소천,58.0,1,0.0,6.0,100,drop,99,100,...,0.234167,0.550415,3.570833,3.570833,"(90, 100]",NaN,False,0,3.570833,1.00
93554,2021-12-19T02:23:40.839Z,주왕,59.0,1,0.0,1.0,100,drop,99,100,...,0.166111,1.290970,5.489611,5.493056,"(90, 100]",NaN,False,0,5.489611,1.00
107414,2021-12-19T08:31:18.894Z,이리,100.0,1,19800.0,1.0,100,drop,99,100,...,0.075000,1.274074,3.586667,3.586667,"(90, 100]",NaN,False,0,3.586667,1.00
112859,2021-12-19T11:10:43.875Z,라이즈,100.0,1,0.0,12.0,100,drop,99,100,...,0.180278,2.827427,5.593778,5.927222,"(90, 100]",NaN,False,0,5.593778,0.94
132065,2021-12-20T03:46:22.528Z,잘될기야,63.0,1,0.0,1.0,100,drop,99,100,...,0.002500,38.777778,6.270000,6.270000,"(90, 100]",NaN,False,0,6.270000,1.00
132560,2021-12-20T04:18:40.112Z,두야,100.0,1,3300.0,0.0,100,drop,99,100,...,0.135833,1.024540,2.990833,2.990833,"(90, 100]",NaN,False,0,2.990833,1.00


## 2Stage Regression

In [23]:
data_dir='levelups_panel_1004_final.csv'
data=pd.read_csv(data_dir, encoding=False)
binge=data['rounded_ratio'].quantile(0.80)
data['binge'] = data.apply(lambda row: 1 if row['rounded_ratio'] >= binge else 0, axis=1)

In [22]:
data.columns

Index(['player_slug', 'character_slug', 'first_session_level', 'binger',
       'total_purchase', 'total_ad', 'highest_level', 'status', 'level_from',
       'level_to', 'event_datetime', 'death', 'ad_revival', 'ad_item',
       'ad_total', 'purchase', 'event_datetime_prev', 'interval',
       'interval_cum', 'playtime_boss', 'playtime_dungeon', 'playtime_tower',
       'playtime_pvp', 'playtime_farm', 'playtime_survival', 'playtime_side',
       'date_event', 'date_event_prev', 'same_date', 'date_interval', 'binge',
       'prev_interval', 'interval_ratio', 'interval_until', 'realtime_tot',
       'level_to_by_10', 'time_diff', 'more_than_3_hours', 'total_num_played',
       'interval_tot', 'rounded_ratio'],
      dtype='object')

In [24]:
columns_to_drop = ['level_from', 'event_datetime','event_datetime_prev','interval_cum','date_event','date_event_prev','same_date'
                   ,'date_interval','prev_interval','interval_until','level_to_by_10','time_diff']


In [25]:
# 컬럼 드랍
data_dropped = data.drop(columns=columns_to_drop)

# 결과 확인
data_dropped.head()

,player_slug,character_slug,first_session_level,binger,total_purchase,total_ad,highest_level,status,level_to,death,...,playtime_farm,playtime_survival,playtime_side,binge,interval_ratio,realtime_tot,more_than_3_hours,total_num_played,interval_tot,rounded_ratio
0,2021-11-01T10:41:53.949Z,화요,119.0,1,0.0,28.0,139,complete,2,0.0,...,0.0,0.0,0.0,1,1.000000,3.726111,False,0,3.726111,1.0
1,2021-11-01T10:41:53.949Z,화요,119.0,1,0.0,28.0,139,complete,3,0.0,...,0.0,0.0,0.0,1,0.202703,3.726111,False,0,3.726111,1.0
2,2021-11-01T10:41:53.949Z,화요,119.0,1,0.0,28.0,139,complete,4,0.0,...,0.0,0.0,0.0,1,1.466667,3.726111,False,0,3.726111,1.0
3,2021-11-01T10:41:53.949Z,화요,119.0,1,0.0,28.0,139,complete,5,0.0,...,0.0,0.0,0.0,1,1.090909,3.726111,False,0,3.726111,1.0
4,2021-11-01T10:41:53.949Z,화요,119.0,1,0.0,28.0,139,complete,6,0.0,...,0.0,0.0,0.0,1,2.958333,3.726111,False,0,3.726111,1.0
